In [1]:
from lxml import etree
import socket
import sys
import binascii
import crc16
import crcmod
import crcmod.predefined
from binascii import unhexlify


In [2]:
def get_data():
    doc = etree.parse('/path/to/file/config.xml')
    root = doc.getroot()
    
    client= root.find('ClientId')
    client_id = client.text
    panel= root.find('SOLID')
    panel_id = panel.text
    s = root.find('Server_IP')
    server_ip = s.text
    p = root.find('Server_Port')
    port = p.text
    seq_no = "0001"
    rev_no = "000001"
    base_msg = '"SIA-DCS"' + seq_no + "R" + rev_no + "L" + client_id + "#" + panel_id + "[#" + panel_id + "|PWR001|V3.01|]_18:14:07,02-07-2020"
    return base_msg


In [3]:
def string_to_send():
    base_msg = '"SIA-DCS"0001R000001L000123#180102[#180102|CMA031|1|0|/path/of/image]_18:14:07,02-07-2020'.upper()
    #print(base_msg)
    base_length = '00'+ str(hex(len(base_msg)).split('x')[-1]) + base_msg
    #print(base_length)
    crc16 = crcmod.mkCrcFun(0x18005, rev=True, initCrc=0x0000, xorOut=0x0000)
    crc = hex(crc16(str(base_msg).encode()))
    s = crc.split('x')[-1] + base_length
    string_send = s.upper()
    msgg = "\n" + string_send + "\r"
    message = msgg.encode()
    print(message)
    
    return message

In [4]:
def send_to_server():
    message = string_to_send()
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server_ip = 10.0.0.0
    port=1234
    server_address = (server_ip, port)
    print(sys.stderr, 'connecting to %s port %s' % server_address)
    sock.connect(server_address)
    sock.sendall(message)
    d = sock.recv(1024)
    print(d.decode("utf-8"))
    return d

In [5]:
d = send_to_server()


b'\n45A60059"SIA-DCS"0001R000001L000123#180102[#180102|CMA031|1|0|/PATH/OF/IMAGE]_18:14:07,02-07-2020\r'
<ipykernel.iostream.OutStream object at 0x7fd75b740710> connecting to 10.0.0.157 port 8092

24930032"ACK"0001R000001L000123#180102[]_16:08:48,02-12-2020

